In [14]:
import itertools
from operator import itemgetter
import subprocess
import os
import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString
import networkx as nx
import osmnx as ox

These set of functions preprocess the informal settlements and POIs layers, reprojecting them and adding the geometries we will need to calculate the distances

In [15]:
def format_informal_settlements_layer(informal_settlements):
    informal_settlements = informal_settlements.to_crs('epsg:3857')
    informal_settlements['is_centroid'] = informal_settlements.centroid
    return(informal_settlements)

def preprocess_poi_layer(pois, municipalities, informal_settlements):
    pois = pois.sjoin(municipalities[['geometry', 'MPIO_CCNCT']], how='left').drop(['index_right'], axis=1)   
    pois = pois.to_crs('epsg:3857')
    pois['buffer'] = pois.buffer(2000)
    pois = pois.set_geometry('buffer')
    pois = pois.sjoin(informal_settlements, how='left')
    pois = pois.set_geometry('geometry')
    pois = pois[~pd.isna(pois.index_right)]
    return(pois)

def download_from_bucket(bucket, path_dst):
    if not os.path.exists(path_dst):
        cmd = f"gsutil cp -n  {bucket} {path_dst}"
        subprocess.run(cmd, shell=True)
    else:
        print("File exists")

These functions perform calculations based on the graph, associated with the geometry of the municipality. The `shortest_path_OSM` function returns the path in LineString format (to calculate the length afterwards).

In [16]:
def elevation_from_nearest_node(geom, graph):
    node = ox.distance.nearest_nodes(graph, geom.x, geom.y)
    return(G.nodes[node]['elevation'])


def graph_from_geometry(geometry):
    bounds = geometry.bounds
    G = ox.graph_from_bbox(bounds[3],bounds[1],bounds[2],bounds[0], network_type='drive')
    return(G)

def add_elevation_to_graph(graph, dem_path):
    G = ox.elevation.add_node_elevations_raster(graph, dem_path, cpus=1)
    assert not np.isnan(np.array(G.nodes(data="elevation"))[:, 1]).any()
    G = ox.elevation.add_edge_grades(G, add_absolute=True)
    return(G)

def shortest_path_OSM(orig_geom, dest_geom, graph, weight=None):
    try:
        orig_node  = ox.distance.nearest_nodes(graph, orig_geom.x, orig_geom.y)
        dest_node  = ox.distance.nearest_nodes(graph, dest_geom.x, dest_geom.y)
        path = nx.shortest_path(graph, orig_node, dest_node, weight)
        route_line = LineString([Point(graph.nodes[n]['x'], graph.nodes[n]['y']) for n in path])
    except:
        route_line = None
    return(route_line)

In [17]:
!ls '/Users/federico/Documents/dymaxion_labs/immap/final/wash/final/'

asentamientos_2018.gpkg     dymaxion_labs.gpkg-shm
asentamientos_2018.gpkg-shm dymaxion_labs.gpkg-wal
asentamientos_2018.gpkg-wal wash_all.gpkg
dymaxion_labs.gpkg


Paths and key variable names are defined in this step.

In [18]:
informal_settlements_path = '/Users/federico/Documents/dymaxion_labs/immap/final/wash/final/dymaxion_labs.gpkg'
is_file_name = 'dl'

municipalities_path = '/Users/federico/Documents/dymaxion_labs/immap/wash/test_magangue/munis_v3.gpkg'

poi_path = '/Users/federico/Documents/dymaxion_labs/immap/wash/test_magangue/hospitals.gpkg'
key_poi = 'CodigoHabi'


municipalities = gpd.read_file(municipalities_path)
pois = gpd.read_file(poi_path)
informal_settlements = gpd.read_file(informal_settlements_path)

In [19]:
gdf_informal_settlements = format_informal_settlements_layer(informal_settlements)
gdf_informal_settlements.head()

,Name,FolderPa,Shape_Le,Shape_Ar,Id_Settl,DPTO_NAM,MPIO_NAM,FUENTE,geometry,is_centroid
0,CARTAGENA_15,12_07_22_TOTALES_consolidados.kml/TOTALES,0,0,CO13001D015,BOLIVAR,CARTAGENA,DYMAXION LABS,"MULTIPOLYGON (((-8409902.080 1159848.107, -840...",POINT (-8409767.509 1159875.761)
1,CARTAGENA_12,12_07_22_TOTALES_consolidados.kml/TOTALES,0,9,CO13001D012,BOLIVAR,CARTAGENA,DYMAXION LABS,"MULTIPOLYGON (((-8413474.467 1156172.684, -841...",POINT (-8413432.330 1156212.996)
2,CARTAGENA_14,12_07_22_TOTALES_consolidados.kml/TOTALES,0,0,CO13001D014,BOLIVAR,CARTAGENA,DYMAXION LABS,"MULTIPOLYGON (((-8403744.955 1165820.843, -840...",POINT (-8403898.518 1165874.084)
3,CARTAGENA_13,12_07_22_TOTALES_consolidados.kml/TOTALES,0,6,CO13001D013,BOLIVAR,CARTAGENA,DYMAXION LABS,"MULTIPOLYGON (((-8403348.828 1165840.274, -840...",POINT (-8403329.129 1165915.600)
4,CARTAGENA_11,12_07_22_TOTALES_consolidados.kml/TOTALES,0,3,CO13001D011,BOLIVAR,CARTAGENA,DYMAXION LABS,"MULTIPOLYGON (((-8409938.114 1157366.727, -840...",POINT (-8409955.887 1157454.437)


In [20]:
gdf_pois = preprocess_poi_layer(pois, municipalities, gdf_informal_settlements)
gdf_pois.head()

,OBJECTID,CodigoHabi,Naturaleza,Nombre,Direccion,NombrePres,TipoZona,Barrio,Telefono,NOM_DPTO,...,index_right,Name,FolderPa,Shape_Le,Shape_Ar,Id_Settl,DPTO_NAM,MPIO_NAM,FUENTE,is_centroid
0,1,520010111401,2,HNAS. HOSPITALARIAS DEL SAGDO. CORAZON DE JESU...,KR 33 # 5 OESTE 104,HNAS. HOSPITALARIAS DEL SAGDO. CORAZON DE JESU...,1,EL BOSQUE,7235684 - 7235685,Nariño,...,220.0,PASTO_7,PASTO_CHACHAGUI-SAMANIEGO.kml/PASTO_CHACHAGUI-...,0.0,0.0,CO52001D007,NARI�O,PASTO,DYMAXION LABS,POINT (-8605446.788 135281.353)
0,1,520010111401,2,HNAS. HOSPITALARIAS DEL SAGDO. CORAZON DE JESU...,KR 33 # 5 OESTE 104,HNAS. HOSPITALARIAS DEL SAGDO. CORAZON DE JESU...,1,EL BOSQUE,7235684 - 7235685,Nariño,...,219.0,PASTO_6,PASTO_CHACHAGUI-SAMANIEGO.kml/PASTO_CHACHAGUI-...,0.0,0.0,CO52001D006,NARI�O,PASTO,DYMAXION LABS,POINT (-8605388.497 136244.198)
1,2,520010145718,1,CENTRO DE SALUD SAN VICENTE,CL 5 KR 35 ESQUINA,EMPRESA SOCIAL DEL ESTADO PASTO SALUD E.S.E.,1,SAN VICENTE,7230132,Nariño,...,220.0,PASTO_7,PASTO_CHACHAGUI-SAMANIEGO.kml/PASTO_CHACHAGUI-...,0.0,0.0,CO52001D007,NARI�O,PASTO,DYMAXION LABS,POINT (-8605446.788 135281.353)
1,2,520010145718,1,CENTRO DE SALUD SAN VICENTE,CL 5 KR 35 ESQUINA,EMPRESA SOCIAL DEL ESTADO PASTO SALUD E.S.E.,1,SAN VICENTE,7230132,Nariño,...,219.0,PASTO_6,PASTO_CHACHAGUI-SAMANIEGO.kml/PASTO_CHACHAGUI-...,0.0,0.0,CO52001D006,NARI�O,PASTO,DYMAXION LABS,POINT (-8605388.497 136244.198)
2,4,520010009601,2,HOSPITAL SAN RAFAEL DE PASTO,CALLE 15 No 42C-35,HOSPITAL SAN RAFAEL DE PASTO,1,SAN JUAN DE DIOS,7235144,Nariño,...,220.0,PASTO_7,PASTO_CHACHAGUI-SAMANIEGO.kml/PASTO_CHACHAGUI-...,0.0,0.0,CO52001D007,NARI�O,PASTO,DYMAXION LABS,POINT (-8605446.788 135281.353)


Iterate through municipalities to avoid duplication in SRTM downloading.

In [ ]:
pois_final = pd.DataFrame()
for i, row in municipalities.iterrows():
    region = row['REGION']
    province_id = row['DPTO_COD']
    municipio_id = row['COD_MUNICIPIO']
    print(municipio_id)
    #POIs within the municipality
    pois_test = gdf_pois[gdf_pois.MPIO_CCNCT==row['MPIO_CCNCT']]
    if pois_test.shape[0]>0:
        #DEM
        os.makedirs('./srtm', exist_ok=True)
        dem_path = f"./srtm/{municipio_id}.tif"
        #create graph
        geometry = row.geometry
        G = graph_from_geometry(geometry)
        G = add_elevation_to_graph(G, dem_path)
        G = ox.project_graph(G)
        CRS = G.graph["crs"]
        #reproject vectorial data based on graph's CRS.
        pois_test['geometry'] =  pois_test['geometry'].to_crs(CRS)
        pois_test['is_centroid'] =  pois_test['is_centroid'].to_crs(CRS)
        #add elevation
        pois_test['elevation_orig'] = pois_test.apply(lambda x: elevation_from_nearest_node(x.geometry, G), axis=1)
        pois_test['elevation_is'] = pois_test.apply(lambda x: elevation_from_nearest_node(x.is_centroid, G), axis=1)
        #shortest path and distance calculation
        pois_test['path'] = pois_test.apply(lambda x: shortest_path_OSM(x.geometry, x.is_centroid, G, "elevation"), axis=1)
        pois_test['path_lenght'] = pois_test['path'].apply(lambda x: gpd.GeoDataFrame(geometry=[x]).length.values[0])
        #reproject to epsg:4326
        pois_final = pd.concat([pois_final,pois_test.sort_values('path_lenght').groupby(key_poi).head(1).reset_index(drop=True)], axis=0)
        pois_test['geometry'] =  pois_test['geometry'].to_crs('epsg:4326')
        pois_test['is_centroid'] =  pois_test['is_centroid'].to_crs('epsg:4326')
        pois_test['buffer'] =  pois_test['buffer'].to_crs('epsg:4326')


8001
File exists


/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set 

3055.437844782573
8573
File exists


/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set 

2659.0695582287112
8758
File exists


...

2222.0299018760807
44001
File exists


/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set 

2011.5448277734438
47001
52051
52079
52240
File exists


/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/federico/opt/anaconda3/envs/OSMNX/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set 

1140.5809307027644
52207
52250
File exists


In [ ]:
pois_final.tail()

Save databases considering different geometries that allows other analysis

In [ ]:
poi_name = 'hospitals'
gpd.GeoDataFrame(pois_final, geometry = pois_final['geometry']).to_crs('epsg:4326').drop(['buffer','path', 'is_centroid'], axis=1)[[key_poi, 'geometry', 'index_right', 'elevation_orig', 'elevation_is']].to_file(f'{poi_name}_{is_file_name}.gpkg')


In [ ]:
gpd.GeoDataFrame(pois_final, geometry = pois_final['path']).set_crs(CRS).to_crs('epsg:4326').drop(['path', 'buffer', 'is_centroid'], axis=1)[[key_poi, 'geometry', 'index_right', 'elevation_orig', 'elevation_is', 'path_lenght']].to_file(f'{poi_name}_{is_file_name}_paths.gpkg')


In [ ]:
gpd.GeoDataFrame(pois_final, geometry = pois_final['buffer']).to_crs('epsg:4326').drop(['path', 'buffer', 'is_centroid'], axis=1)[[key_poi, 'geometry', 'index_right','elevation_orig', 'elevation_is']].to_file(f'{poi_name}_{is_file_name}_buffer.gpkg')
